## Data Tools Engineer Task:

In [38]:
import pandas as pd
from itertools import permutations, product
import openpyxl
from datetime import date
import requests
import json

## Load CarsData

In [39]:
data_sheet = 'CarsData.xlsx'
df = pd.read_excel(data_sheet, sheet_name='Sheet1', index_col=0).T # transpose the data

## Display data columns (testing if it loaded right)

In [40]:
df.columns.values

array(['Q1-IsElectric', 'Q2-KM', 'Q3-EngineSize', 'Q4-Color',
       'Q5-ModelData'], dtype=object)

## API call

In [41]:
url = 'https://api.coinbase.com/v2/exchange-rates'
x = requests.get(url, auth=('myAPIkey', 'CLP'))
api_data = json.loads(x.text)

In [42]:
usd_rate = api_data['data']['rates']['USD']
clp_rate = api_data['data']['rates']['CLP']
exhange_rate = abs(float(usd_rate) / float(clp_rate))

## clean data and parase the columns

In [43]:
cars_df = df.drop("Condition") # we dont need the condition

In [44]:
# parasing the columns data
def parse_att(data):
    return data.iloc[0].split(';')

In [45]:
Q1 = parse_att(cars_df['Q1-IsElectric'])
Q2 = parse_att(cars_df['Q2-KM'])
Q3 = parse_att(cars_df['Q3-EngineSize'])
Q4 = parse_att(cars_df['Q4-Color'])
Q5 = parse_att(cars_df['Q5-ModelData'])

## create permutations cars data from the columns data

In [46]:
wb_out = openpyxl.Workbook()
ws_out = wb_out.active
for i in product(Q1, Q2, Q3, Q4, Q5):
    ws_out.append(i)
wb_out.save(filename='carsCleaned.xlsx')

## Load new data to Pandas DB

In [47]:
cars = pd.read_excel('carsCleaned.xlsx',header=None,
                     names=['Q1-IsElectric', 'Q2-KM', 'Q3-EngineSize', 'Q4-Color', 'Q5-ModelData'])

## check conditons

In [48]:
cond = df.loc['Condition']

In [49]:
print(cond)

Property Name
Q1-IsElectric          NaN
Q2-KM                  NaN
Q3-EngineSize    Q1==False
Q4-Color               NaN
Q5-ModelData           NaN
Name: Condition, dtype: object


## clean data by condition

In [50]:
cars.loc[cars['Q1-IsElectric'] == False, 'Q3-EngineSize'] = None

## Test new data ( we should get 600rows  and engine size nan if q1 is false )

In [51]:
cars.tail(10)

,Q1-IsElectric,Q2-KM,Q3-EngineSize,Q4-Color,Q5-ModelData
590,False,10000000,NaN,Black,"\n{\n ""brand"": ""Tesla"",\n ""model"": ""Model 3""..."
591,False,10000000,NaN,Red,"{\n ""brand"": ""Ford"",\n ""model"": ""Mustang"",\n..."
592,False,10000000,NaN,Red,"\n{\n ""brand"": ""BMW"",\n ""model"": ""M3"",\n ""y..."
593,False,10000000,NaN,Red,"\n{\n ""brand"": ""Tesla"",\n ""model"": ""Model 3""..."
594,False,10000000,NaN,White,"{\n ""brand"": ""Ford"",\n ""model"": ""Mustang"",\n..."
595,False,10000000,NaN,White,"\n{\n ""brand"": ""BMW"",\n ""model"": ""M3"",\n ""y..."
596,False,10000000,NaN,White,"\n{\n ""brand"": ""Tesla"",\n ""model"": ""Model 3""..."
597,False,10000000,NaN,Yellow,"{\n ""brand"": ""Ford"",\n ""model"": ""Mustang"",\n..."
598,False,10000000,NaN,Yellow,"\n{\n ""brand"": ""BMW"",\n ""model"": ""M3"",\n ""y..."
599,False,10000000,NaN,Yellow,"\n{\n ""brand"": ""Tesla"",\n ""model"": ""Model 3""..."


In [52]:
cars.head(10)

,Q1-IsElectric,Q2-KM,Q3-EngineSize,Q4-Color,Q5-ModelData
0,True,100,1.0,Black,"{\n ""brand"": ""Ford"",\n ""model"": ""Mustang"",\n..."
1,True,100,1.0,Black,"\n{\n ""brand"": ""BMW"",\n ""model"": ""M3"",\n ""y..."
2,True,100,1.0,Black,"\n{\n ""brand"": ""Tesla"",\n ""model"": ""Model 3""..."
3,True,100,1.0,Red,"{\n ""brand"": ""Ford"",\n ""model"": ""Mustang"",\n..."
4,True,100,1.0,Red,"\n{\n ""brand"": ""BMW"",\n ""model"": ""M3"",\n ""y..."
5,True,100,1.0,Red,"\n{\n ""brand"": ""Tesla"",\n ""model"": ""Model 3""..."
6,True,100,1.0,White,"{\n ""brand"": ""Ford"",\n ""model"": ""Mustang"",\n..."
7,True,100,1.0,White,"\n{\n ""brand"": ""BMW"",\n ""model"": ""M3"",\n ""y..."
8,True,100,1.0,White,"\n{\n ""brand"": ""Tesla"",\n ""model"": ""Model 3""..."
9,True,100,1.0,Yellow,"{\n ""brand"": ""Ford"",\n ""model"": ""Mustang"",\n..."


In [53]:
cars.shape[0] ## return number of rows (all good)

600

## Create new full Date column with car manufacture date as type dd/mm/yyyy

In [54]:
cars['Dates'] = cars['Q5-ModelData'].apply(
    lambda x: x.split(',')[2].split(':')[1].rstrip('}').rstrip('\n')).astype(int)

In [55]:
cars['Dates'] = pd.to_datetime(cars['Dates'],format='%Y')

## Create new full Date column current date as type dd/mm/yyyy

In [56]:
today = date.today()
cars["Current_Date"] = today

## Create column with the value of days passed since manufacture till today and diplay the data

In [57]:
cars['Days_Passed'] = (pd.to_datetime(cars['Current_Date']) - cars['Dates']).dt.days

In [58]:
cars.head(10)

,Q1-IsElectric,Q2-KM,Q3-EngineSize,Q4-Color,Q5-ModelData,Dates,Current_Date,Days_Passed
0,True,100,1.0,Black,"{\n ""brand"": ""Ford"",\n ""model"": ""Mustang"",\n...",1964-01-01,2021-07-27,21027
1,True,100,1.0,Black,"\n{\n ""brand"": ""BMW"",\n ""model"": ""M3"",\n ""y...",1981-01-01,2021-07-27,14817
2,True,100,1.0,Black,"\n{\n ""brand"": ""Tesla"",\n ""model"": ""Model 3""...",2020-01-01,2021-07-27,573
3,True,100,1.0,Red,"{\n ""brand"": ""Ford"",\n ""model"": ""Mustang"",\n...",1964-01-01,2021-07-27,21027
4,True,100,1.0,Red,"\n{\n ""brand"": ""BMW"",\n ""model"": ""M3"",\n ""y...",1981-01-01,2021-07-27,14817
5,True,100,1.0,Red,"\n{\n ""brand"": ""Tesla"",\n ""model"": ""Model 3""...",2020-01-01,2021-07-27,573
6,True,100,1.0,White,"{\n ""brand"": ""Ford"",\n ""model"": ""Mustang"",\n...",1964-01-01,2021-07-27,21027
7,True,100,1.0,White,"\n{\n ""brand"": ""BMW"",\n ""model"": ""M3"",\n ""y...",1981-01-01,2021-07-27,14817
8,True,100,1.0,White,"\n{\n ""brand"": ""Tesla"",\n ""model"": ""Model 3""...",2020-01-01,2021-07-27,573
9,True,100,1.0,Yellow,"{\n ""brand"": ""Ford"",\n ""model"": ""Mustang"",\n...",1964-01-01,2021-07-27,21027


# Create column with the exhange rate between currencies

In [59]:
cars['exhange_rate'] = exhange_rate

In [60]:
cars.head(10)

,Q1-IsElectric,Q2-KM,Q3-EngineSize,Q4-Color,Q5-ModelData,Dates,Current_Date,Days_Passed,exhange_rate
0,True,100,1.0,Black,"{\n ""brand"": ""Ford"",\n ""model"": ""Mustang"",\n...",1964-01-01,2021-07-27,21027,0.001316
1,True,100,1.0,Black,"\n{\n ""brand"": ""BMW"",\n ""model"": ""M3"",\n ""y...",1981-01-01,2021-07-27,14817,0.001316
2,True,100,1.0,Black,"\n{\n ""brand"": ""Tesla"",\n ""model"": ""Model 3""...",2020-01-01,2021-07-27,573,0.001316
3,True,100,1.0,Red,"{\n ""brand"": ""Ford"",\n ""model"": ""Mustang"",\n...",1964-01-01,2021-07-27,21027,0.001316
4,True,100,1.0,Red,"\n{\n ""brand"": ""BMW"",\n ""model"": ""M3"",\n ""y...",1981-01-01,2021-07-27,14817,0.001316
5,True,100,1.0,Red,"\n{\n ""brand"": ""Tesla"",\n ""model"": ""Model 3""...",2020-01-01,2021-07-27,573,0.001316
6,True,100,1.0,White,"{\n ""brand"": ""Ford"",\n ""model"": ""Mustang"",\n...",1964-01-01,2021-07-27,21027,0.001316
7,True,100,1.0,White,"\n{\n ""brand"": ""BMW"",\n ""model"": ""M3"",\n ""y...",1981-01-01,2021-07-27,14817,0.001316
8,True,100,1.0,White,"\n{\n ""brand"": ""Tesla"",\n ""model"": ""Model 3""...",2020-01-01,2021-07-27,573,0.001316
9,True,100,1.0,Yellow,"{\n ""brand"": ""Ford"",\n ""model"": ""Mustang"",\n...",1964-01-01,2021-07-27,21027,0.001316


## Calculate the price of the car and return the data as excel

In [61]:
cars['Price'] = cars['Q2-KM'] * cars['Days_Passed'] * cars['exhange_rate'] 

In [62]:
cars.to_excel("carsPriced.xlsx", index=False, columns=['Q1-IsElectric', 'Q2-KM', 'Q3-EngineSize', 'Q4-Color',
       'Q5-ModelData','Price'])  

## display cars price data 

In [63]:
cars_priced = pd.read_excel('carsPriced.xlsx')

In [65]:
cars_priced.tail(10)

,Q1-IsElectric,Q2-KM,Q3-EngineSize,Q4-Color,Q5-ModelData,Price
590,False,10000000,NaN,Black,"\n{\n ""brand"": ""Tesla"",\n ""model"": ""Model 3""...",7.540664e+06
591,False,10000000,NaN,Red,"{\n ""brand"": ""Ford"",\n ""model"": ""Mustang"",\n...",2.767147e+08
592,False,10000000,NaN,Red,"\n{\n ""brand"": ""BMW"",\n ""model"": ""M3"",\n ""y...",1.949913e+08
593,False,10000000,NaN,Red,"\n{\n ""brand"": ""Tesla"",\n ""model"": ""Model 3""...",7.540664e+06
594,False,10000000,NaN,White,"{\n ""brand"": ""Ford"",\n ""model"": ""Mustang"",\n...",2.767147e+08
595,False,10000000,NaN,White,"\n{\n ""brand"": ""BMW"",\n ""model"": ""M3"",\n ""y...",1.949913e+08
596,False,10000000,NaN,White,"\n{\n ""brand"": ""Tesla"",\n ""model"": ""Model 3""...",7.540664e+06
597,False,10000000,NaN,Yellow,"{\n ""brand"": ""Ford"",\n ""model"": ""Mustang"",\n...",2.767147e+08
598,False,10000000,NaN,Yellow,"\n{\n ""brand"": ""BMW"",\n ""model"": ""M3"",\n ""y...",1.949913e+08
599,False,10000000,NaN,Yellow,"\n{\n ""brand"": ""Tesla"",\n ""model"": ""Model 3""...",7.540664e+06
